# One symbol program
## Run an entire cycle upto df_nakeds for one symbol
- [x] Get chains
- [x] Get a symbol
- [x] Update its chain strikes
- [x] Get its closest SD
- [ ] Get its margin
- [ ] Get its expPrice

In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'.upper()
symbol = 'SPY'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util
util.startLoop()

In [ ]:
# Imports 
from utils import Vars, get_pickle

# Set the root
from from_root import from_root
ROOT = from_root()

_vars = Vars(MARKET)
PORT = port = _vars.PORT
OPT_COLS = _vars.OPT_COLS[0]
COLS = _vars.COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()


# One symbol chain

In [ ]:
# Imports
from utils import get_pickle, make_chains, get_strike_closest_to_und, get_margins, make_a_raw_contract, qualify_naked_opts, get_exchange_name
from ib_insync import Option
import asyncio

In [ ]:
# Get one symbol
UNDS_PATH = DATAPATH / 'unds.pkl'

# Make chains for one symbol
contract = get_pickle(UNDS_PATH)[symbol]


In [ ]:
# Get all the chains of the symbol
df_sym_chain = asyncio.run(make_chains(contracts=contract))

# Remove fractional strikes from df_sym_chain
only_whole = df_sym_chain.strike.apply(lambda x: x%1==0)
df_sym_chain = df_sym_chain[only_whole]

# Get the closest strikes to und
df_ch = get_strike_closest_to_und(df_sym_chain, 1)

In [ ]:
# Get margins and commissions from the options closest to und
df_chq = qualify_naked_opts(df_ch)

In [17]:
# SNP conditions for `mgnCompute`

# ... margin_compute is 30% of value
gt_16_67 = df_chq.undPrice > 16.67

# ... margin_compute is 30% of $5 per share
gt_5_lt_16_67 = (df_chq.undPrice > 5) & (df_chq.undPrice <= 16.67)

# ... margin_compute is 100% of stock value
lt_5 = (df_chq.undPrice > 2.5) & (df_chq.undPrice <= 5)

# ... margin_compute is 100% of $2.5
lt_2_5 = (df_chq.undPrice <= 2.5)


In [30]:
def gt_16_67(df: pd.DataFrame) -> dict:
    """Computes margin for undPrice > 16.67. It is 30% of value"""

    pct = 0.3

    gt_16_67 = df.undPrice > 16.67
    return dict(zip(df[gt_16_67].symbol, df[gt_16_67].undPrice * df[gt_16_67].lot * df[gt_16_67].multiplier * pct))

In [ ]:
def gt_5_lt_16_67(df: pd.DataFrame) -> dict:
    """Computes margin for 5 > undPrice >= 16.67"""

    pct = 0.3*5 # 30% of $5 per share

    gt_5_lt_16_67 = (df_chq.undPrice > 5) & (df_chq.undPrice <= 16.67)

    return dict(zip(df[gt_5_lt_16_67].symbol, df[gt_5_lt_16_67].lot * df[gt_5_lt_16_67].multiplier * pct))



In [31]:
from utils import get_pickle
get_pickle(DATAPATH / 'df_und_prices.pkl')

,secType,conId,symbol,price,iv
0,STK,9720,MMM,107.87,0.232968
1,STK,4065,ABT,111.51,0.200016
2,STK,118089500,ABBV,177.50,0.183263
3,STK,67889930,ACN,345.03,0.189971
4,STK,265768,ADBE,511.25,0.285920
...,...,...,...,...,...
237,IND,137851301,XSP,524.15,0.104484
238,IND,416863,OEX,2477.85,0.107482
239,IND,12793724,XEO,2477.85,0.103117
240,IND,13455763,VIX,12.92,0.711937


In [ ]:
# Compute margins from get_unds_with_prices.
# TODO: Integrate `lots` and `multiplier` to output of `get_unds_with_prices`
# TODO: Modify get_unds_with_prices to put the lot and multiplier fields.

In [37]:
from utils import get_unds_with_prices
get_unds_with_prices(MARKET)

Mkt prices with IVs: 100%|██████████| 6/6 [00:54<00:00,  9.06s/it]


,secType,conId,symbol,price,iv
0,STK,9720,MMM,91.400,0.294685
1,STK,4065,ABT,109.225,0.248582
2,STK,118089500,ABBV,162.435,0.283688
3,STK,67889930,ACN,315.600,0.227321
4,STK,265768,ADBE,475.470,0.284604
...,...,...,...,...,...
236,IND,12793724,XEO,2467.760,NaN
237,IND,13455763,VIX,14.910,2.455503
238,IND,416888,RUT,NaN,0.216991
239,STK,756733,SPY,511.700,NaN


In [35]:
from ib_insync import Stock, IB

from utils import qualify_me
import asyncio

with IB().connect(port=PORT) as ib:
    y = asyncio.run(qualify_me(ib=ib, contracts = Stock(symbol='QQQ', exchange='NYSE', currency='USD')))

Qualifying contracts: 100%|██████████| 1/1 [00:00<00:00,  1.94it/s]


In [36]:
y

[Stock(conId=320227571, symbol='QQQ', exchange='NYSE', primaryExchange='NASDAQ', currency='USD', localSymbol='QQQ', tradingClass='NMS')]

In [ ]:


df_chq.loc[gt_16_67, 'mgnCompute'] = df_chq[gt_16_67].undprice * df_chq[gt_16_67].lot * 0.3


In [ ]:
# Get the chain with the smallest dte
df_ch_min_dte = df_chq[df_chq.dte == df_chq.dte.min()].reset_index(drop=True)
df_ch_min_dte


In [ ]:
# make raw target options and qualify them
from utils import raw_target_opts
df_raw_opts = raw_target_opts(df_chains, 1)


In [ ]:
df_raw_opts

In [ ]:
df_opts = qualify_naked_opts(df_raw_opts)

In [ ]:
df_opts

In [ ]:
# get expected price for the target options
from utils import nakeds_with_expPrice
df_n = nakeds_with_expPrice(df_opts)

In [ ]:
df_n.head(3)

In [ ]:
3.45*50/99697.63*365/11.344245